In [1]:
# Load model from pickle file
import random
import numpy as np
import matplotlib.pyplot as plt
import pickle
import copy
import sys
from micrograd.engine import Value
from micrograd.nn import Neuron, Layer, MLP
import cvxpy as cp
from micrograd.ibp import ibp, Interval

np.random.seed(1337)
random.seed(1337)


with open('model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)




# Collect all ReLU nodes in the computation graph of the last score
def collect_relu_nodes(output_node, node_bounds, splitted_nodes=()):
    relu_nodes = []
    for v in output_node.compute_graph():
        if v.op == 'ReLU' and v not in splitted_nodes:
            # Only add if lower and upper bounds have a sign change
            relu_input = list(v.prev)[0]
            lower, upper = node_bounds[v]
            if lower * upper < 0:
                relu_nodes.append(v)
    return relu_nodes

  

def branch_and_bound(score, in_bounds, splits={}):
    
    # ibp
    node_bounds = ibp(score, in_bounds | splits, return_all=True)
    
    
    
    # Collect ReLU nodes
    relu_nodes = collect_relu_nodes(score, node_bounds, splits.keys())
    print("test")
    # No ReLU nodes with sign change in bounds found
    if not relu_nodes:
        #print("No ReLU nodes with sign change found, returning bounds directly.")
        return node_bounds[score]
    
    # Pick a ReLU node at random to branch on
    #chosen_relu = random.choice(relu_nodes)
    chosen_relu = relu_nodes[0]  # For deterministic behavior, use the first one
    relu_input = list(chosen_relu.prev)[0]
    
    #print(f"Chosen ReLU node: {chosen_relu}, input: {relu_input}")  # debugging
    
    
    # Branch 1: ReLU input >= 0
    
    split1 = copy.copy(splits)
    split1[relu_input] = Interval(0, node_bound[relu_input].upper)
    
    # Check if the bounds are valid for the first branch via Planet relaxation
    check1_l, check1_u = planet_relaxation(score, in_bounds ,node_bounds | split1)
    if check1_l == float('inf') or check1_u == float('-inf'):
        bounds1 =  float('inf'), float('-inf')
    elif check1_l >= 0:
        bounds1= check1_l, check1_u
    elif check1_u < 0:
        return check1_l, check1_u
    else:
        bounds1 = branch_and_bound(score, in_bounds, split1)

    # Branch 2: ReLU input <= 0
    
    split2 = copy.copy(splits)
    split2[relu_input] = Interval(node_bound[relu_input].lower, 0)
    
    check2_l, check2_u = planet_relaxation(score, in_bounds, node_bounds | split2)
    if check2_l == float('inf') or check2_u == float('-inf'):
        bounds2 = float('inf'), float('-inf')
    elif check2_l >= 0:
        bounds2= check2_l, check2_u
    elif check2_u < 0:
        return check2_l, check2_u
    else:
        bounds2 = branch_and_bound(score, in_bounds, split2)
    

    # Return global bounds
    print (f"bounds1: {bounds1}, bounds2: {bounds2}")  # debugging
    return min(bounds1[0], bounds2[0]), max(bounds1[1], bounds2[1])
   

def planet_relaxation(output: Value, in_bounds, node_bounds):
    env = {}  # maps Value nodes to cp.Variable or float
    constraints = []

    # Traverse in topological order
    for v in output.compute_graph():
        if len(v.prev) == 0:
            # Input node
            if (v in in_bounds):
                #alternavit to lower != data 
                #(v.lower == -0.1 and v.upper == 0.1) or (v.lower == 0.4 and v.upper == 0.6)
                var = cp.Variable()
                env[v] = var
                constraints += [
                    var >= node_bounds[v].lower,
                    var <= node_bounds[v].upper,
                ]
            else:
                # Constant/weight node
                #print(f"Assigning constant: {v}, value type: {type(v.data)}") #debugging
                env[v] = v.data
        else:
            # Operation node
            if v.op == "+":
                a, b = [env[p] for p in v.prev]
                var = cp.Variable()
                constraints.append(var == a + b)
                env[v] = var
            elif v.op == "*":
                # For PLANET, only allow multiplication by constant (affine layers)
                a, b = [env[p] for p in v.prev]
                #print(f"Multiplying types: {type(a)}, {type(b)}") #debugging
                
                if isinstance(a, (int, float)) and isinstance(b, (int, float)):
                    raise NotImplementedError("PLANET relaxation does not support multiplication of two constants.")
                else:
                    var = cp.Variable()
                    constraints.append(var == a * b)
                    env[v] = var
                
            elif v.op == "ReLU":
                inp = [env[p] for p in v.prev][0]
                var = cp.Variable()
                # Get input bounds for relaxation
                input_node = list(v.prev)[0]
                l = node_bounds[input_node].lower
                u = node_bounds[input_node].upper
                # Standard PLANET ReLU relaxation
                if u <= 0:
                    # If upper bound is non-positive, ReLU is always zero
                    constraints.append(var == 0)
                elif l >= 0:
                    # If lower bound is non-negative, ReLU is the input
                    constraints.append(var == inp)
                else:
                    constraints += [
                        var >= 0,
                        var >= inp,
                        var <= (u / (u - l)) * (inp - l) if u > l else var <= 0,
                        var <= u if u > 0 else var <= 0,
                    ]
                env[v] = var
            else:
                raise NotImplementedError(f"Operation {v.op} not supported in PLANET relaxation.")

    prob_lower = cp.Problem(cp.Minimize(env[output]), constraints)
    result_lower = prob_lower.solve()
    
    prob_upper = cp.Problem(cp.Maximize(env[output]), constraints)
    result_upper = prob_upper.solve()
    
    return result_lower , result_upper



In [3]:
with open('model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)
    
x = [Value(2), Value(-3)]
out = loaded_model(x)
print(f"Output: {out.data}")

Output: 8.177270077410343


In [4]:
with open('model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

x = [Value(0), Value(0.5)]
in_bounds = {xi: Interval(xi.data - 0.1, xi.data + 0.1) for xi in x}
out = loaded_model(x)
score = ibp(out, in_bounds)
print(f"Score: {score}")


Score: Interval(lower=np.float64(-2.6787537647437722), upper=np.float64(1.3256224613907477))


In [15]:
with open('model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

x = [Value(0), Value(0.5)]
in_bounds = {xi: Interval(xi.data - 0.1, xi.data + 0.1) for xi in x}
out = loaded_model(x)


lower_bound, upper_bound = branch_and_bound(out, in_bounds)
print(f"Lower bound: {lower_bound}, Upper bound: {upper_bound}")


test
Lower bound: -2.6787537647437722, Upper bound: 1.3256224613907477
